In [1]:
import QuantLib as ql
from matplotlib import pyplot as plt
import numpy as np
import math

%matplotlib inline

In [ ]:
todaysDate = ql.Date(15, ql.May, 2023)
ql.Settings.instance().evaluationDate = todaysDate

## Option construction
exercise = ql.AmericanExercise(todaysDate, ql.Date(17, ql.May, 2024))
payoff = ql.PlainVanillaPayoff(ql.Option.Put, 40.0)

option = ql.VanillaOption(payoff, exercise)


# ### Market data
#
underlying = ql.SimpleQuote(36.0)
dividendYield = ql.FlatForward(todaysDate, 0.00, ql.Actual365Fixed())
volatility = ql.BlackConstantVol(todaysDate, ql.TARGET(), 0.20, ql.Actual365Fixed())
riskFreeRate = ql.FlatForward(todaysDate, 0.06, ql.Actual365Fixed())

#
process = ql.BlackScholesMertonProcess(
    ql.QuoteHandle(underlying),
    ql.YieldTermStructureHandle(dividendYield),
    ql.YieldTermStructureHandle(riskFreeRate),
    ql.BlackVolTermStructureHandle(volatility),
)

# ### Pricing
#
# We'll collect tuples of method name, option value, and estimated error from the analytic formula.

results = []

#### Analytic approximations
option.setPricingEngine(ql.BaroneAdesiWhaleyApproximationEngine(process))
results.append(("Barone-Adesi-Whaley", option.NPV()))

option.setPricingEngine(ql.BjerksundStenslandApproximationEngine(process))
results.append(("Bjerksund-Stensland", option.NPV()))

#### Finite-difference method
timeSteps = 801
gridPoints = 800

option.setPricingEngine(ql.FdBlackScholesVanillaEngine(process, timeSteps, gridPoints))
results.append(("finite differences", option.NPV()))

# Longstaff Schwartz
mc_engine = ql.MCAmericanEngine(
    process,
    "pseudorandom",
    timeSteps=timeSteps,
    timeStepsPerYear=None,
    antitheticVariate=False,
    controlVariate=False,
    requiredSamples=None,
    requiredTolerance=None,
    maxSamples=None,
    seed=0,
    polynomOrder=2,
    polynomType=ql.LsmBasisSystem.Monomial,
    nCalibrationSamples=2048,
    antitheticVariateCalibration=None,
    seedCalibration=None
)

option.setPricingEngine(mc_engine)

results.append(("LS American", option.NPV()))

# #### Li, M. QD+ American engine
qdplus_engine =  ql.QdPlusAmericanEngine(
    process,
    interpolationPoints = 8,
    solverType = ql.QdPlusAmericanEngine.Halley,
    eps = 1e-6,
    maxIter = None
)
option.setPricingEngine(qdplus_engine)
results.append(("QD+", option.NPV()))

# #### Leif Andersen, Mark Lake and Dimitri Offengenden high performance American engine

option.setPricingEngine(
    ql.QdFpAmericanEngine(process, ql.QdFpAmericanEngine.accurateScheme())
)
results.append(("QD+ fixed point", option.NPV()))

In [3]:
results

[('Barone-Adesi-Whaley', 4.463536264085715),
 ('Bjerksund-Stensland', 4.4569588667986935),
 ('finite differences', 4.489992153451754),
 ('QD+', 4.501016922307475),
 ('QD+ fixed point', 4.490552145592267)]